# Import

In [1]:
# Verify which python environment we are running the notebook on

!pyenv version
!pip install kaggle
!pip install matplotlib


wino (set by PYENV_VERSION environment variable)


In [ ]:
#packages import

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import re
#from kaggle.api.kaggle_api_extended import KaggleApi
#import kaggle
import os

os.environ['KAGGLE_USERNAME'] = 'sabrinaauger'
os.environ['KAGGLE_KEY'] = '8db79b4ccae98a2c74af940e1f1b7c35'
os.getcwd()

In [ ]:
#Loading Kaggle username and key from user's own .kaggle/kaggle.json

def load_kaggle_credentials():
    kaggle_creds_path = os.path.expanduser("~/.kaggle/kaggle.json")

    if os.path.exists(kaggle_creds_path):
        with open(kaggle_creds_path) as f:
            kaggle_creds = json.load(f)
            return kaggle_creds.get('username'), kaggle_creds.get('key')

    return None, None

# Load Kaggle credentials
kaggle_username, kaggle_key = load_kaggle_credentials()

# Authenticate with Kaggle API
api = KaggleApi()
api.authenticate()

#REMOVE AFTER TESTING: Print Kaggle username for testing
print(f"Authenticated as: {kaggle_username}")


# Import the dataset from Kaggle

In [149]:
#Retrieve the data from Kaggle platform
!kaggle datasets download -d zynicide/wine-reviews


100%|██████████████████████████████████████| 50.9M/50.9M [00:33<00:00, 1.93MB/s]
100%|██████████████████████████████████████| 50.9M/50.9M [00:33<00:00, 1.58MB/s]


In [ ]:
#Unzip the data zip file
!unzip wine-reviews.zip

In [ ]:
#Retrieve the data from csv unzipped files into two dataframes
wine1_df=pd.read_csv("~/code/sabrinaauger/wino/raw_data/winemag-data-130k-v2.csv")


In [ ]:
wine2_df=pd.read_csv("~/code/sabrinaauger/wino/raw_data/winemag-data_first150k.csv")


In [ ]:
wine2_df.head()


In [ ]:
wine1_df.head()


# Data vizualisation

In [ ]:
#Columns overview of both datasets
wine1_df.columns, wine2_df.columns


In [ ]:
#Analysing the variety column and the unique value
wine1_df['variety'].unique()


In [ ]:
# Looking at the description of 1 specific wine
index_of_record_to_view = 3
des = wine2_df.loc[index_of_record_to_view, 'description']
des


In [ ]:
wine1_df.shape, wine2_df.shape


# Preprocessing the data 

## Cleaning wine1_df and wine2_df dataset 

### Removing Unuseful columns for the recommendation system

In [ ]:
#Remove both columns taster name and taster twitter_handle as they are not relevant for our analysis
wine_review1_df=wine1_df.drop(columns=["taster_twitter_handle","taster_name","Unnamed: 0"])
wine_review2_df=wine2_df.drop(columns=["Unnamed: 0"])
#wine_review2_df.head()


In [ ]:
#Extracting the year from the column title in wine1_df

# Extract the year from title into a dedicated column using regular expression
#regular instruction (\d{4}) simply looks for four consecutive digits in the strings
wine_review1_df['Year']=wine_review1_df['title'].str.extract(r'(\d{4})')

missing_year=wine_review1_df['Year'].isna().sum()
print(f'Missing data in year variable : {missing_year} - Decision taken: not adding this additional variable')
wine_review1_df.drop(columns='Year', inplace=True)


### Duplicates

In [ ]:
#checking if there is any overall duplicates in both datasets
print(wine_review1_df.duplicated().sum())
print(wine_review2_df.duplicated().sum())
print("Note - TODO: Check the duplicates 'title' after cleaning the missing data")


In [ ]:
### Checking at the duplicate
duplicates=wine_review1_df[wine_review1_df.duplicated(subset=('title'))]
duplicates_sorted = duplicates.sort_values(by=['title'])
#duplicates_sorted
#should not be considered as duplicates - cleaning the data first


duplicates=wine_review2_df[wine_review2_df.duplicated(subset=('designation'))]
duplicates_sorted = duplicates.sort_values(by=['designation'])
#duplicates_sorted
#need to clean the data to interpretate the duplicates


### Missing data 

In [ ]:
#Overall missing data in wine_review1_df dataset
wine_review1_df.isna().sum(),wine_review2_df.isna().sum()


#### Variety missing data cleaning : 

In [ ]:
#Variety missing data
wine_review1_df[wine_review1_df['variety'].isna()]
#Only one record is missing in wine_review1_df and most of the data for this wine is missing - Decided to remove the record
wine_review1_df = wine_review1_df.dropna(subset=['variety'])

#As title contains already the information of designation column - decision : drop designation
#wine_review1_df=wine_review1_df.drop(columns=['designation'])

#print('Decision taken: Dropped the only record wine with missing variety as the record had only missing data')
#print('Decision taken: Dropped column designation as title contains already the information')


#### Designation missing data cleaning

In [ ]:
m1=wine_review1_df['designation'].isna().sum()
m2=wine_review2_df['designation'].isna().sum()
print(f'Designation missing in wine_review1_df:{m1},in wine_review2_df:{m2}')


##### Wine_review1_df

In [ ]:
#Filling the missing designation using title variable

wine_review1_df['designation'].fillna(wine_review1_df['title'], inplace=True)
def remove_year(designation):
    pattern = r'\b\d{4}\b'  # Regex pattern to match four-digit year
    return re.sub(pattern, '', designation).strip()

# Remove year from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_year)

#Dealing with the parentheses values in designation
def remove_parentheses_text(designation):
    pattern = r'\s*\([^)]*\)'  # Regex pattern to match text within parentheses
    return re.sub(pattern, '', designation).strip()

# Remove text within parentheses from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_parentheses_text)


##### Wine_review2_df

In [ ]:
#As no title provide the designation of the wine the missing designation in wine_review2_df must be dropped
wine_review2_df.dropna(subset='designation', inplace=True)


In [ ]:
wine_review1_df.isna().sum()


#### Region missing data cleaning
Analysing the region_1 and region_2 variables and missing data:

##### Wine_review1_df

In [ ]:
wine_review1_df.head()


In [ ]:
#Analysing the region_1 and region_2 variables
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review1_df['region_1'].fillna(wine_review1_df['region_2'], inplace=True)
wine_review1_df['region_1'].isna().sum()


In [ ]:
#Removing the region_2 column from the dataset
wine_review1_df.drop(columns='region_2', inplace=True)


In [ ]:
wine_review1_df.isna().sum()


In [ ]:
#import re #import re module for regular expressions
#filling the missing region with the region displayed in the title

#Function to extract the content within parentheses from title column
def extract(s):
    if isinstance(s, str):
        extracted = re.search(r'\((.*?)\)', s)
        if extracted:
            return extracted.group(1)
    return None
wine_review1_df['region_1'].fillna(wine_review1_df['title'].apply(extract), inplace=True)

wine_review1_df[wine_review1_df['region_1'].isna()]

#412 instances don't have a region displayed but have other useful data available


- Filling the missing region_1 data when applicable using title

In [ ]:
#checking the data that have both missing province and region
check = wine_review1_df['region_1'].isnull() & wine_review1_df['province'].isnull()
# Filter the DataFrame based on the condition
rows_missing_region_province = wine_review1_df[check]
#rows_missing_region_province

wine_review1_df['region_1'].fillna(wine_review1_df['province'], inplace=True)


In [ ]:
wine_review1_df['region_1'].isna().sum()


In [ ]:
#wine_review1_df[wine_review1_df['region_1'].isna()]
#dropping the 63 remaining instances that don't have country , province and region filled
wine_review1_df.dropna(subset='region_1', inplace=True)


##### Wine_review2_df

In [ ]:
#Analysing the region_1 and region_2 variables
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review2_df['region_1'].fillna(wine_review2_df['region_2'], inplace=True)
wine_review2_df['region_1'].isna().sum()


In [ ]:
#Removing the region_2 column from the dataset
wine_review2_df.drop(columns='region_2', inplace=True)


In [ ]:

wine_review2_df['region_1'].fillna(wine_review2_df['province'], inplace=True)
wine_review2_df[wine_review2_df['region_1'].isna()]
wine_review2_df.dropna(subset='region_1', inplace=True)


### Missing province variable

##### Wine_review1_df

In [ ]:
#Displaying the missing data
wine_review1_df[wine_review1_df['province'].isna()]
wine_review1_df.shape[0]


In [ ]:
#Trying to fill the missing province with the title information

def extract_province_from_title(title):
    pattern = r'\((.*?)\)'  # Regex pattern to match text within parentheses
    match = re.search(pattern, title)
    if match:
        return match.group(1)
    else:
        return None

# Extract province from 'Title' column
wine_review1_df['extracted_Province'] = wine_review1_df['title'].apply(extract_province_from_title)

# Fill missing 'Province' values with extracted province information
wine_review1_df['province'].fillna(wine_review1_df['extracted_Province'], inplace=True)
wine_review1_df.drop(columns=['extracted_Province'], inplace=True)


In [ ]:
wine_review1_df.isna().sum()


##### Wine_review2_df

In [ ]:
wine_review2_df.isna().sum()


### Missing price
Decision taken : Missing price filled with mean price of wines with same variety

##### Wine_review1_df

In [ ]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review1_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review1_df.groupby('variety')['price'].mean())
mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable

wine_review1_df = pd.merge(wine_review1_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review1_df['price'] = wine_review1_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [ ]:
wine_review1_df=wine_review1_df.drop(columns=['price_mean'])


In [ ]:
wine_review1_df['price']=wine_review1_df['price'].round(2)
#wine_review1_df


In [ ]:
wine_review1_df=wine_review1_df.dropna()
wine_review1_df.isna().sum()


##### Wine_review2_df

In [ ]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review2_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review2_df.groupby('variety')['price'].mean())
mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable

wine_review2_df = pd.merge(wine_review1_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review2_df['price'] = wine_review1_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [ ]:
wine_review2_df=wine_review2_df.drop(columns=['price_mean'])


In [ ]:
wine_review2_df['price']=wine_review2_df['price'].round(2)
#wine_review1_df


In [ ]:
wine_review2_df=wine_review2_df.dropna()
wine_review2_df.isna().sum()

In [ ]:
wine_review1_df.columns, wine_review2_df.columns

#### Removing the variable title from wine_review1_df 
Not relevant as other variables already contain the info in title 

In [ ]:
wine_review1_df.drop(columns='title', inplace=True)
wine_review2_df.drop(columns='title', inplace=True)


### Merging both datasets

In [ ]:
wine_review1_df.isna().sum(), wine_review2_df.isna().sum()


In [ ]:
wine_reviews_df=pd.concat([wine_review1_df, wine_review2_df])


In [ ]:
wine_reviews_df.shape


In [ ]:
wine_reviews_df.isna().sum()
wine_reviews_df.head()


###  Outliers and anomalies



#### Numerical data : Identifying outliers 
- if >480k euros => most expensive bottle in the world
- if >100 for score

In [ ]:
from scipy import stats

# Assuming df is your DataFrame
numerical_columns = wine_reviews_df.select_dtypes(include=['float64', 'int64']).columns
numerical_columns

In [ ]:
# Finding outliers
threshold_point=100
threshold_price=480000
outliers_point = wine_reviews_df[(abs(wine_reviews_df['points']) > threshold_point)]
outliers_price = wine_reviews_df[(abs(wine_reviews_df['price'])> threshold_price)]


In [ ]:
#Displaying the outliers
outliers_point
outliers_price
#No price or points outliers


=> No price or points outliers 

#### Categorical data : Identifying outliers 

In [ ]:
import stat

#wine_reviews_df.info()
categorical_columns=wine_reviews_df.select_dtypes(include='object').columns


In [ ]:
categorical_columns

##### Country

In [ ]:
# Set max rows to display to a high number to show all unique values
pd.set_option('display.max_rows', None)

wine_reviews_df['country'].value_counts()
#No issue

##### Designation and variety
- Both columns should be merged as they provide better understanding of the wine variety together - new variable created

In [ ]:
wine_reviews_df['designation'].value_counts()
#Designation column has false values and is an additional information given by the wine variety variable
#decision merging both column as unique wine variety variable
wine_reviews_df['wine_variety']=wine_reviews_df['designation']+' '+wine_reviews_df['variety']

In [ ]:
#Removing repeated word in wine_variety created column

def remove_repeated_words(text):
    words = text.split()
    unique_words = []
    count = {}
    
    for word in words:
        count[word] = count.get(word, 0) + 1
        if count[word] <= 2:
            unique_words.append(word)
    
    return ' '.join(unique_words)

# Apply the function to the DataFrame column
wine_reviews_df['wine_variety'] = wine_reviews_df['wine_variety'].apply(remove_repeated_words)


In [ ]:
# Delete both columns designation and variety after columns merge

wine_reviews_df

#####  Province

In [ ]:
wine_reviews_df['province'].value_counts()
# No outliers or anonalies found 

In [ ]:
wine_reviews_df.head()

In [ ]:
wine_reviews_df['winery'].value_counts()
# No winery outliers or anomalies found 


#### Create wine type dictionary

In [ ]:
unique_variety=wine_reviews_df['wine_variety'].unique()
#pd.set_option('display.max_rows', None)


In [ ]:
list(unique_variety)

In [69]:
variety_to_type={}


def assign_type(variety):
    red_varieties = ['Cabernet Sauvignon', 'Merlot', 'Pinot Noir', 'Syrah','Sirah', 'Zinfandel', 'Malbec', 'Red',"Nero d'Avola",'Noir'
                    ]
    white_varieties = ['Chardonnay', 'Riesling', 'Sauvignon Blanc', 'Pinot Grigio', 'Gewürztraminer','Bianco', 
                       'White','White Blend', 'Gris','Blanc','Viognier']
    rose_varieties=['Rose','Rosé','Rosato','']
    sparkling_varieties = ['Champagne', 'Prosecco', 'Cava','Sparkling']

    if any(variety.str.contains(var) for var in white_varieties):
        return 'White'
    elif any(variety.str.contains(var) for var in red_varieties):
        return 'Red'
    elif any(variety.str.contains(var) for var in sparkling_varieties:
        return 'Sparkling'
    else:
        return 'Other' 


for variety in unique_varieties:
    # Manually assign types based on the variety and on the logic built in the assign_type function
    variety_to_type[variety] = assign_type(variety) 